In [1]:
import json
import os
import collections.abc
import csv
from tabulate import tabulate

def get_metrics(res_dir):
  with open(f"{res_dir}/trainlog.csv") as f:
    reader = csv.reader(f)
    keys =next(reader)
    metrics = []
    
    for row in reader:
      metrics.append({k:float(v) for k,v in zip(keys, row)})
  return metrics

def get_accs_at_epoch(m, epoch, test_acc=None):
  accs = {}
  for k in ['accuracy', 'val_accuracy', 'test_accuracy']:
    if k in m:
      accs[k] = round(m[k][epoch] * 100, 1)
  if test_acc is not None and 'test_accuracy' not in accs:
    accs['test_accuracy'] = round(test_acc * 100, 1)
  
  return accs

def get_results(week):
  results = []
  for name in os.listdir(f"{week}"):
    if name.startswith("-"):
      continue
    for seed in os.listdir(f"{week}/{name}"):
      if seed.startswith("-"):
        continue
      item = {
        "name": name,
        'week': week,
        'seed': seed[4:]
      }

      
      if os.path.exists(f"{week}/{name}/{seed}/config.json"):
        with open(f"{week}/{name}/{seed}/config.json") as f:
          config = json.load(f)

        item.update(config)
        
      
      if os.path.exists(f"{week}/{name}/{seed}/overview.json"):
        with open(f"{week}/{name}/{seed}/overview.json") as f:
          overview = json.load(f)
        
        test_accuracy = overview.get("test_accuracy", None)
        if isinstance(test_accuracy, collections.abc.Sequence):
          test_accuracy = test_accuracy[0]

        item.update({"test_accuracy": test_accuracy, 'train_time': round(overview.get("train_time", 0) / (60*60), 2)})
      
      if os.path.exists(f"{week}/{name}/{seed}/trainlog.csv"):
        train_log = get_metrics(f"{week}/{name}/{seed}")
        max_val = train_log['val_accuracy'].index(max(train_log['val_accuracy']))
        accs = get_accs_at_epoch(train_log, max_val, item.get('test_accuracy', None))
        item.update(accs)
        item.update({"best_val_epoch": max_val + 1})
      else: 
        continue

      results.append(item)
  return results

def print_table(data, keys):
  data = [[r.get(k, '') for k in keys] for r in data]
  print(tabulate(data, headers=keys))

keys = ['name', 'week', 'seed', 'accuracy', 'val_accuracy',  'test_accuracy', 'best_val_epoch', 'G_epoch', 'hidden_size', 'batch_size', 'learning_rate', 'dropout', 'train_time',]

def print_week(week):
  results = get_results(week)
  print_table(results, keys)

all_keys = ['name', 'runs', 'accuracy', 'val_accuracy',  'test_accuracy', 'best_val_epoch', 'G_epoch', 'hidden_size', 'batch_size', 'learning_rate', 'dropout', 'train_time',]
average_keys = ['accuracy', 'val_accuracy',  'test_accuracy', 'best_val_epoch']
sum_keys = ['train_time']

def print_week_average(week):
  results = get_results(week)
  names = set([r['name'] for r in results])
  aggregates = []
  for name in names:
      seeds = [r for r in results if r['name'] == name]
      item = seeds[0]
      item['runs'] = len(seeds)
      for k in average_keys:
        vals = [s[k] for s in seeds if k in s]
        if(len(vals) > 0):
          item[k] = round(sum([r for r in vals]) / len(vals),1)
      for k in sum_keys:
        vals = [s[k] for s in seeds if k in s]
        if(len(vals) > 0):
          item[k] = round(sum([r for r in vals]),1)
          
      aggregates.append(item)
  print_table(aggregates, all_keys)

In [2]:
import wandb

In [3]:
def import_wandb(dir):
  if os.path.exists(f"{dir}/config.json"):
    with open(f"{dir}/config.json") as f:
      config = json.load(f)
      #print(config)
        
      if os.path.exists(f"{dir}/overview.json"):
        with open(f"{dir}/overview.json") as f:
          overview = json.load(f)
        
        test_accuracy = overview.get("test_accuracy", None)
        if isinstance(test_accuracy, collections.abc.Sequence):
          overview['test_accuracy'] = test_accuracy[0]
      else:
        print("no overview",dir)
        return

      
      if os.path.exists(f"{dir}/trainlog.csv"):
        train_log = get_metrics(f"{dir}")
        
        light_ajrnn = config.get('light_ajrnn', False)

        wandb.init(
          project="light-ajrnn" if light_ajrnn else "ajrnn",
          config=config,
        )
        for l in train_log:
          wandb.log(l)
        
        overview_keys = ['test_accuracy', 'train_time', 'test_time', 'best_val_epoch']
        overview = {k: overview[k] for k in overview_keys if k in overview}
        wandb.log(overview)
        wandb.finish()

In [ ]:
import_wandb("week50/G-LIGHT-GRU128-G1-LR7-B256-REG-DROP/see196") # imac

In [19]:
week = "week48"
for name in os.listdir(f"{week}"):
    if name.startswith("-"):
      continue
    for seed in os.listdir(f"{week}/{name}"):
      if seed.startswith("-"):
        continue
      import_wandb(f"{week}/{name}/{seed}")

accuracy,▁▃▄▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████
best_val_epoch,▁
c_loss,█▆▅▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
d_loss,█████████▇▇▇▅▃▃▂▂▂▁▂▂▁▁▂▁▂▁▁▁▁▁▂▂▁▁▁▂▂▂▃
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
g_loss,▂▃▃▃▃▃▃▃▇▆▆▅▃▄▄█▆▇▆▇▅▇▇█▇▇████▇▄▄▅▄▄▃▃▁▁
imp_loss,▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████
reg_loss,█▇▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅
test_accuracy,▂▆▇██▇▇▇▆▆▆▆▆▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_c_loss,▃▂▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████
test_d_loss,████▇▇▇▇▆▇▇▇▆▄▄▂▂▁▁▁▁▁▁▂▁▁▃▂▁▁▁▃▂▂▂▂▂▃▄▄


no overview week48/GRU128-G1-LR6-BS32/seed89


accuracy,▁▃▅▆▇▇▇▇▇▇▇█████████████████████████████
best_val_epoch,▁
c_loss,█▆▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
d_loss,█▄▄▄▄▄▅▅▅▅▅▅▅▅▅▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▄▅▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
g_loss,▆███▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▄▃▃▂▁▁
imp_loss,▁▁▂▄▄▄▄▄▅▆▆▅▅▆▇▇███▇▇▇▇▆▅▆▆▅▅▅▅▅▅▅▆▆▅▄▄▄
reg_loss,▄▂▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇██
test_accuracy,███▇▇▆▄▄▅▄▁▁▁▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▆▅▅▅▆▆▆▆▆▆▆▆
test_c_loss,▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▆▆▅▅▅▅▅▅▅▆▆▆▅▅▅▆█▇▇████
test_d_loss,█▆▅▅▆▆▆▆▆▆▆▆▆▆▆▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▁▁▁▂▄▄▅


no overview week48/GRU128-G5-LR6-BS32-REG-DROP/seed89


accuracy,▁▂▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
best_val_epoch,▁
c_loss,█▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
d_loss,█▆▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
g_loss,▁▄█████▇▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄
imp_loss,▅▆███▇▆▆▅▅▄▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
reg_loss,██▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
test_accuracy,▁▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
test_c_loss,█▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_d_loss,▅▃▁▂▁▂▃▄▄▅▅▅▅▅▅▅▅▅▆▇▇▇▇▇▇▇██████████████


accuracy,▁▂▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇███████████████████████
best_val_epoch,▁
c_loss,█▇▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
d_loss,▁█▆▇▆▇▇▇▇▇▇▇▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▅▆▆▆▆▆
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
g_loss,█▁▃▁▃▁▁▁▁▁▁▁▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▄▃▃▃▃▃
imp_loss,▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
reg_loss,██▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
test_accuracy,▁▄▇▇▇▇▇▇████████████████████████████████
test_c_loss,█▇▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_d_loss,█▁▃▂▃▂▂▂▂▂▂▂▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▄▃▃▃▃▃


accuracy,▁▃▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
best_val_epoch,▁
c_loss,█▆▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
d_loss,█▆▂▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
g_loss,▁▃▇█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
imp_loss,▁▁▁▁▁▁▁▂▂▂▂▂▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇████████
reg_loss,█▇▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃
test_accuracy,▁▂▄▆▇███████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▄▄▄▃▂▂▁
test_c_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▄▄▅▅
test_d_loss,█▄▂▁▁▁▁▁▂▂▂▃▃▃▂▂▂▂▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆


accuracy,▁▃▄▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████
best_val_epoch,▁
c_loss,█▆▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
d_loss,▁█▄▄▃▃▃▃▃▄▃▃▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
g_loss,█▁▂▂▃▃▃▃▃▂▃▃▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
imp_loss,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇██
reg_loss,█▇▆▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄
test_accuracy,▁▅▆▆▇▇▇▇█████████████▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅
test_c_loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂
test_d_loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


no overview week48/GRU128-G1-LR3-BS256-REG/seed196
